# OpenAI가 제공하는 프롬프트 엔지니어링 모범사례

OpenAI 모델에 명확하고 효과적인 지침을 제공하는 방법을 설명합니다.

source: https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-openai-api

## 프롬프트 엔지니어링 작동 방식
OpenAI 모델이 훈련되는 방식으로 인해 특히 잘 작동하고 보다 유용한 [모델](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models) 출력으로 이어지는 특정 프롬프트 형식이 있습니다. [​OpenAI의 공식 프롬프트 엔지니어링 가이드](https://platform.openai.com/docs/guides/prompt-engineering)는 일반적으로 프롬프트 팁을 학습하기 좋은 시작점입니다. 아래에는 잘 작동한다고 생각되는 여러 가지 프롬프트 형식이 제시되어 있지만 작업에 더 적합할 수 있는 다양한 형식을 자유롭게 탐색해 보세요.

## 사용법에 대한 예시
### 참고
> {text input here} 은 실제 텍스트/컨텍스트를 입력하는 표시입니다.

In [1]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT","").strip(),
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("OPENAI_API_VERSION")
)

deployment_name = os.getenv('DEPLOYMENT_NAME')

## 1. 최신 모델 사용

최상의 결과를 얻으려면 최신 모델을 사용하십시오. 최신 모델은 엔지니어에게 프롬프트를 사용하는 것이 더 쉬울 수 있습니다.
최신 모델 정보는 [다음 링크](https://learn.microsoft.com/ko-kr/azure/ai-services/openai/concepts/models)에서 확인 가능합니다.

## 2. 프롬프트 시작 부분에 지침(instructions)을 넣고 ### 또는 """를 사용하여 지침(instruction)과 컨텍스트(context)를 구분합니다.

### 덜 효과적인 방법 ❌:  
>Summarize the text below as a bullet point list of the most important points.  
>  
>{text input here}
----  
### 더 효과적인 방법 ✅:  
>Summarize the text below as a bullet point list of the most important points.  
>  
>Text: """  
>{text input here}.  
>"""

In [2]:
response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Summarize the text below as a bullet point list of the most important points. \n\n \
        ###\n\nWe’re happy to announce that OpenAI and Microsoft are extending our partnership.\
        This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
        in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
        increasingly safe, useful, and powerful. \n\n \
        In pursuit of our mission to ensure advanced AI benefits all of humanity, OpenAI remains a \
        capped-profit company and is governed by the OpenAI non-profit. This structure allows us to \
        raise the capital we need to fulfill our mission without sacrificing our core beliefs about \
        broadly sharing benefits and the need to prioritize safety. \
        Microsoft shares this vision and our values, and our partnership is instrumental to our progress. \n###',}],
        max_tokens=400,)

print(response.choices[0].message.content)

- OpenAI and Microsoft are extending their partnership
- Microsoft is making a multi-billion dollar investment in OpenAI
- The investment will support independent research and development of safer and more powerful AI
- OpenAI is a capped-profit company governed by the OpenAI non-profit to ensure benefits are shared broadly
- OpenAI's mission is to ensure advanced AI benefits all of humanity
- Microsoft shares OpenAI's vision and values to prioritize safety and broad benefit-sharing
- The partnership with Microsoft is essential for OpenAI's progress


## 3. 원하는 맥락, 결과, 길이, 형식, 스타일 등에 대해 구체적이고 설명적이며 최대한 상세하게 작성하십시오.
### 덜 효과적인 방법 ❌:  
>Write a poem about OpenAI. 
----
### 더 효과적인 방법 ✅:  
>Write a short inspiring poem about OpenAI, focusing on the recent DALL-E product launch (DALL-E is a text to image ML model) in the style of a {famous poet}

In [3]:
response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a poem about OpenAI.',}],
        max_tokens=400,)

print(response.choices[0].message.content)

Innovation springs from depths unknown,
A marvel of machines that have grown,
OpenAI, with brilliance shone,
A powerful force, intelligence honed.

Through algorithms and lines of code,
It learns and adapts, in genius mode,
Unleashing potential, a vast abode,
Exploring realms where minds have strode.

In the realm of AI, it stands tall,
Pushing boundaries, breaking the wall,
Unlocking doors to visions all,
OpenAI, on a daring call.

With every task, it strives to excel,
In a world where wonders dwell,
A beacon of progress, we can tell,
OpenAI, a tale to retell.


In [4]:
response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a short inspiring poem about OpenAI, \
                focusing on the recent DALL-E product launch in the style of Ernest Hemingway',}],
        max_tokens=400,)

print(response.choices[0].message.content)

In the realm of AI so vast and grand,
DALL-E emerges, a creative hand.
Imagining art with a digital eye,
In OpenAI's realm, creativity will fly.

A new frontier in tech's illustrious quest,
DALL-E dazzles, pushing limits to the test.
Ernest Hemingway would surely agree,
In OpenAI's journey, greatness we see.


## 4. 예제를 통해 원하는 출력 형식을 명시합니다.
### 덜 효과적인 방법 ❌:  
> Extract the entities mentioned in the text below. Extract the following 4 entity types: company names, people names, specific topics and themes.  
>  
> Text: {text}  
----  
### 더 효과적인 방법 ✅:  

In [5]:
response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Extract the companyn names then years in the following text below and output start index and end index of each entity.\
                Generate output as {"text": "OpenAI", "start": 28, "end": 34} \
                ###\
                We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n \
                ###\
                ',}],
        max_tokens=400,)

print(response.choices[0].message.content)

Here are the extracted company names along with their start and end indexes:

1. {"text": "OpenAI", "start": 28, "end": 34}
2. {"text": "Microsoft", "start": 52, "end": 61}


보여주고 말하세요 - 특정 형식 요구 사항이 표시되면 모델이 더 잘 반응합니다. 또한 이를 통해 프로그래밍 방식으로 여러 출력을 안정적으로 구문 분석하는 것이 더 쉬워졌습니다.

In [6]:
response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Extract the entities mentioned in the text below. \
                Extract the important entities mentioned in the text below. \
                First extract all company names, then extract all years, \
                then extract specific topics which fit the content and finally extract general overarching themes\n\n \
                Desired format: \
                Company names: <comma_separated_list_of_company_names> \
                Years: -||- \
                Specific topics: -||- \
                General themes: -||- \
                """\
                We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n \
                """\
                ',}],
        max_tokens=400,)

print(response.choices[0].message.content)

Company names: OpenAI, Microsoft  
Years: 2019, 2021  
Specific topics: Partnership, Research, AI development  
General themes: Collaboration, Innovation, AI safety


## 5. Start with zero-shot, then few-shot (example), neither of them worked, then fine-tune 

In [7]:
# zero-shot

response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":"You are a helpful assistant. Extract keywords from the corresponding texts below."},
                {"role":"user","content": 'Text: \n\
            We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
            This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
            in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
            increasingly safe, useful, and powerful. \n\nKeywords:    ',}],
        max_tokens=400,)

print(response.choices[0].message.content)

OpenAI, Microsoft, partnership, multi-year, investment, billion dollar, 2019, 2021, independent research, AI, safe, useful, powerful.


In [8]:
# few-shot

response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":"You are a helpful assistant. Extract keywords from the corresponding texts below.\n\n \
                Text: Stripe provides APIs that web developers can use to integrate \
                payment processing into their websites and mobile applications. \
                Keywords: Stripe, payment processing, APIs, web developers, websites, mobile applications \
                ###\n\
                Text: OpenAI has trained cutting-edge language models that are very good at understanding \
                and generating text. Our API provides access to these models and can be used to solve virtually \
                any task that involves processing language. \n\
                Keywords: language models, text processing, API.\n\n\
                ##W"},
                {"role":"user","content": '\n\
                Text: We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n\
                Keywords:',}],
        max_tokens=400,)

print(response.choices[0].message.content)

OpenAI, Microsoft, partnership, investment, AI, research.


## 6. 애매한 표현과 부정확한 설명을 줄입니다

In [9]:
# 애매하고 부정확한 설명

response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a description for a new product. This product is a new generation of car seat. \
                The description for this product should be fairly short, a few sentences only, and not too much more.',}],
        max_tokens=400,)

print(response.choices[0].message.content)

Introducing the latest innovation in automobile safety: the next generation car seat. Designed with cutting-edge technology and superior comfort in mind, this car seat offers unparalleled protection and support for passengers of all ages. Say goodbye to traditional car seats and experience the future of safe and comfortable travel with our revolutionary product.


In [10]:
# 명확한 설명

response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a description for a new product. This product is a new generation of car seat. \
                Use a 3 to 5 sentence paragraph to describe this product.',}],
        max_tokens=400,)

print(response.choices[0].message.content)

Introducing the revolutionary next generation car seat: the SmartRide360. Designed with cutting-edge technology and superior safety features, this car seat ensures ultimate comfort and protection for your little one. Equipped with 360-degree rotation capabilities, adjustable recline settings, and a built-in smart sensor system, the SmartRide360 offers unparalleled convenience for parents on the go. Say goodbye to fumbling with straps and struggling to install the seat - the SmartRide360 makes traveling with your child a breeze. Upgrade to the SmartRide360 and experience the future of car seat innovation today!


## 7. 하지 말아야 할 것을 말하지 말고 대신 해야 할 것을 말하십시오.

In [11]:
# 하지 말아야 할 것에 대한 정의 없을 경우 발생하는 사건

response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'The following is a conversation between an Agent and a Customer. DO NOT ASK USERNAME OR PASSWORD. DO NOT REPEAT. \n\n\
                Customer: I can’t log in to my account.\n\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)

Customer: I can't log in to my account.
Agent: I'm sorry to hear that. Have you tried resetting your password?


In [12]:
# 해야 하는 것에 대한 정의를 하는 경우

response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":'The following is a conversation between an Agent and a Customer. The agent will attempt to diagnose the \
                problem and suggest a solution, whilst refraining from asking any questions related to PII. \
                Instead of asking for PII, such as username or password, refer the user to the help \
                article www.samplewebsite.com/help/faq \n\n\
                Customer: I can’t log in to my account. \n\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)

I'm sorry to hear that you're having trouble logging into your account. For assistance with logging in, please refer to our help article at www.samplewebsite.com/help/faq. This article should provide you with useful information and troubleshooting steps to resolve your login issue. If you continue to experience difficulties, feel free to reach out to our support team for further assistance.


## 8. 코드 생성 - 모델을 특정 패턴으로 시작할 수 있도록 "시작하는 단어"를 사용하세요.

In [13]:
response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":'# Write a simple python function that \n\
                # 1. Ask me for a number in mile\n\
                # 2. It converts miles to kilometers',}],
        max_tokens=400,)

print(response.choices[0].message.content)

Here is a simple Python function that converts miles to kilometers:

```python
def miles_to_km():
    # Ask the user for a number in miles
    miles = float(input("Enter a number in miles: "))

    # Convert miles to kilometers
    km = miles * 1.60934

    print(f"{miles} miles is equal to {km} kilometers.")

miles_to_km()
```

You can copy and paste this function into your Python environment or code editor and run it to convert miles to kilometers. Let me know if you need any further assistance!


In [14]:
# GPT-4-Turbo 모델에서 결과물이 더 잘 나옵니다.

response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":'# Write a simple python function that \n\
                # 1. Ask me for a number in mile\n\
                # 2. It converts miles to kilometers\n\
                 import ',}],
        max_tokens=400,)

print(response.choices[0].message.content)

Here is a simple Python function that asks the user for a number in miles and converts it to kilometers:

```python
def miles_to_kilometers():
    miles = float(input("Enter the distance in miles: "))
    kilometers = miles * 1.60934
    return f"{miles} miles is equal to {kilometers} kilometers."

result = miles_to_kilometers()
print(result)
```

You can copy and paste this code into a Python environment to run the function.
